Lane Detection App

In [2]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np

In [6]:
root = Tk()
root.title("Lane Detection")
root.geometry("1200x450")
root.resizable(0, 0)

# Global variables for video capture and processing
vid = None
is_canny_active = False
is_hough_active = False
is_kalman_active = False
is_sobel_active = False
is_all_in_one_active = False  # Added for the "All-in-One" mode
kalman = cv2.KalmanFilter(4, 2)  # Initialize Kalman filter

# Function to start the camera
def start_camera():
    global vid
    vid = cv2.VideoCapture(1)
    update_camera()

# Function to update the camera feed
def update_camera():
    global vid
    ret, frame = vid.read()
    if ret:
        frame = cv2.resize(frame, (288, 216))
        
        if is_canny_active:
            frame = detect_lanes_canny(frame)
        elif is_hough_active:
            frame = detect_lanes_hough(frame)
        elif is_kalman_active:
            frame = detect_and_track_lanes(frame)
        elif is_sobel_active:
            frame = detect_lanes_sobel(frame)
        elif is_all_in_one_active:  # Added condition for the "All-in-One" mode
            frame = detect_lanes_all_in_one(frame)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        frame_pil = Image.fromarray(frame)
        frame_tk = ImageTk.PhotoImage(frame_pil)
        camera_module_label.configure(image=frame_tk)
        camera_module_label.image = frame_tk
        camera_module_label.after(10, update_camera)

# Function to detect lanes using Canny edge detection
def detect_lanes_canny(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

# Function to detect lanes using Hough line transform
def detect_lanes_hough(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return frame

# Function to detect and track lanes using Kalman filter
def detect_and_track_lanes(frame):
    global kalman
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            center = np.array([x1 + (x2 - x1)/2, y1 + (y2 - y1)/2], dtype=np.float32).reshape(2, 1)
            prediction = kalman.predict()
            measurement = np.array([[np.float32(center[0])], [np.float32(center[1])]])
            kalman.correct(measurement)
            cv2.circle(frame, (int(prediction[0]), int(prediction[1])), 4, (0, 0, 255), -1)
    return frame

# Function to detect lanes using Sobel edge detection
def detect_lanes_sobel(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    sobel_combined = cv2.bitwise_or(np.uint8(abs_sobelx), np.uint8(abs_sobely))
    return cv2.cvtColor(sobel_combined, cv2.COLOR_GRAY2BGR)

# Function to detect lanes using multiple techniques combined (Canny, Hough, Kalman, Sobel)
def detect_lanes_all_in_one(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            center = np.array([x1 + (x2 - x1)/2, y1 + (y2 - y1)/2], dtype=np.float32).reshape(2, 1)
            prediction = kalman.predict()
            measurement = np.array([[np.float32(center[0])], [np.float32(center[1])]])
            kalman.correct(measurement)
            cv2.circle(frame, (int(prediction[0]), int(prediction[1])), 4, (0, 0, 255), -1)

            # Add text "LANE" at the midpoint of the detected lane with adjusted font and color
            cv2.putText(frame, "LANE", (int((x1 + x2) / 2), int((y1 + y2) / 2)), cv2.FONT_HERSHEY_TRIPLEX, 0.5, (204, 255, 229), 2, cv2.LINE_AA)

    return frame

# Load background and labels
background_image = PhotoImage(file="background.png")
background_label = Label(root, image=background_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

Title_label = Label(root, text="Lane Detection using OpenCV", font="Elephant 15 bold", bg='#202930', fg='white')
Title_label.place(x=450, y=12)

app_icon = PhotoImage(file="road.png")
root.iconphoto(False, app_icon)
app_logo = PhotoImage(file="Open_CV.png")
app_logo_resize = app_logo.subsample(5, 5)
app_logo_label = Label(root, image=app_logo_resize, bg='#202930')
app_logo_label.place(x=380, y=0)

camera_module = PhotoImage(file="camera_module.png")
camera_module_resize = camera_module.subsample(2, 2)
camera_module_label = Label(root, image=camera_module_resize, bg='#4da89f', border=3)
camera_module_label.place(x=10, y=100)

###canny-edge
def canny_all():
    toggle_canny_info()
    canny_edge()

# Create a label for Canny Edge Detection information
canny_info_text = """
Canny Edge Detection:\n
Canny edge detection is a popular algorithm for detecting edges in images.\nIt involves multiple steps, including smoothing the image, finding gradients, applying non-maximum suppression, and edge tracking by hysteresis.\nThe result is a binary image highlighting edges. Canny is effective due to its ability to detect edges accurately,\nhandle noise well, and provide thin, well-connected edges.
"""
canny_info_label = Label(root, text=canny_info_text, wraplength=480, justify='left',font="Ariel 10", bg="#06403b",fg="white", borderwidth=1, relief='solid')
canny_info_label.place(x=500,y=140)

# Function to toggle visibility of the Canny Edge Detection label
def toggle_canny_info():
    if canny_info_label.winfo_ismapped():
        canny_info_label.place_forget()  # Hide the label if it's currently visible
    else:
        canny_info_label.place(x=500,y=140)

# Function to toggle Canny edge detection
def canny_edge():
    global is_canny_active, is_hough_active, is_kalman_active, is_sobel_active
    is_canny_active = not is_canny_active
    is_hough_active = False
    is_kalman_active = False
    is_sobel_active = False

# Initially hide the label
canny_info_label.place_forget()

ce_button = Button(root, text="CED", font="Elephant 10", bg='#4da89f', activebackground='#1f534f', fg='#202930',
                   border=2, command=canny_all)
ce_button.place(x=321, y=102)

###sobel_edge
def sobel_all():
    toggle_sobel_info()
    sobel_edge()

# Create a label for Sobel Edge Detection information
sobel_info_text = """
Sobel Edge Detection:\n
Sobel edge detection is a simple yet effective method for finding edges in images.\n It uses convolution with Sobel kernels to calculate the gradient magnitude in the horizontal and vertical directions.\nSobel detects edges by highlighting regions of high gradient intensity, making it suitable for various applications like image segmentation and feature extraction.
"""
sobel_info_label = Label(root, text=sobel_info_text, wraplength=480, justify='left',font="Ariel 10", bg="#06403b",fg="white", borderwidth=1, relief='solid')
sobel_info_label.place(x=500,y=140)

# Function to toggle visibility of the Sobel Edge Detection label
def toggle_sobel_info():
    if sobel_info_label.winfo_ismapped():
        sobel_info_label.place_forget()  # Hide the label if it's currently visible
    else:
        sobel_info_label.place(x=500,y=140)

# Function to toggle Sobel edge detection
def sobel_edge():
    global is_sobel_active, is_canny_active, is_hough_active, is_kalman_active
    is_sobel_active = not is_sobel_active
    is_canny_active = False
    is_hough_active = False
    is_kalman_active = False

# Initially hide the label
sobel_info_label.place_forget()

se_button = Button(root, text="SED", font="Elephant 10", bg='#4da89f', activebackground='#1f534f', fg='#202930',
                   border=2, command=sobel_all)
se_button.place(x=320, y=147)

###kalman filter
def kalman_all():
    toggle_kalman_info()
    kalman_filter()

# Create a label for Kalman Filter information
kalman_info_text = """
Kalman Filter:\n
Kalman filter is a recursive algorithm used for state estimation in dynamic systems subject to uncertain, \npotentially noisy measurements. It predicts the state of a system based on its previous state and a model of the system's dynamics.\nThen, it corrects this prediction using measurements, optimizing estimation accuracy. Kalman filters find applications \nin various fields like navigation, signal processing, and computer vision for tasks such as object tracking and sensor fusion.
"""
kalman_info_label = Label(root, text=kalman_info_text, wraplength=480, justify='left',font="Ariel 10", bg="#06403b",fg="white", borderwidth=1,relief='solid')
kalman_info_label.place(x=500,y=110)

# Function to toggle visibility of the Kalman Filter label
def toggle_kalman_info():
    if kalman_info_label.winfo_ismapped():
        kalman_info_label.place_forget()  # Hide the label if it's currently visible
    else:
        kalman_info_label.place(x=500,y=110)

# Function to toggle Kalman Filter
def kalman_filter():
    global is_kalman_active, is_canny_active, is_hough_active, is_sobel_active
    is_kalman_active = not is_kalman_active
    is_canny_active = False
    is_hough_active = False
    is_sobel_active = False

# Initially hide the label
kalman_info_label.place_forget()

kf_button = Button(root, text="KFR", font="Elephant 10", bg='#4da89f', activebackground='#1f534f', fg='#202930',
                   border=2, command=kalman_all)
kf_button.place(x=319, y=195)

###hough transform
def hough_all():
    toggle_hough_info()
    hough_transform()

# Create a label for Hough Transform information
hough_info_text = """
Hough Transform:\n
Hough transform is a technique used in image processing for detecting simple geometric shapes like lines, circles, or ellipses.\nIt converts points in an image to a parameter space where simple shapes correspond to curves. \nBy detecting peaks in this parameter space, Hough transform identifies lines or other shapes.\nIt's particularly useful in tasks like lane detection, shape recognition, and feature extraction in computer vision applications.
"""
hough_info_label = Label(root, text=hough_info_text, wraplength=480, justify='left',font="Ariel 10", bg="#06403b",fg="white", borderwidth=1,relief='solid')
hough_info_label.place(x=500,y=110)

# Function to toggle visibility of the Hough Transform label
def toggle_hough_info():
    if hough_info_label.winfo_ismapped():
        hough_info_label.place_forget()  # Hide the label if it's currently visible
    else:
        hough_info_label.place(x=500,y=110)

# Function to toggle Hough Transform
def hough_transform():
    global is_hough_active, is_canny_active, is_kalman_active, is_sobel_active
    is_hough_active = not is_hough_active
    is_canny_active = False
    is_kalman_active = False
    is_sobel_active = False

# Initially hide the label
hough_info_label.place_forget()

ht_button = Button(root, text="HTR", font="Elephant 10", bg='#4da89f', activebackground='#1f534f', fg='#202930',
                   border=2, command=hough_all)
ht_button.place(x=318, y=245)

###all in one
def all_all():
    toggle_all_info()
    all_in_one()

# Create a label for all_in_one information
all_info_text = """
All algorithms are incorporated together.\n
"""
all_info_label = Label(root, text=all_info_text, wraplength=480, justify='left',font="Ariel 10", bg="#06403b",fg="white", borderwidth=1, relief='solid')
all_info_label.place(x=500,y=140)

# Function to toggle visibility of the all_in_one label
def toggle_all_info():
    if all_info_label.winfo_ismapped():
        all_info_label.place_forget()  # Hide the label if it's currently visible
    else:
        all_info_label.place(x=500,y=140)

# Function to toggle All-in-One lane detection mode
def all_in_one():
    global is_all_in_one_active
    is_all_in_one_active = not is_all_in_one_active
    if is_all_in_one_active:
        all_button.config(text="STOP", bg='#ff6347')  # Change button text and color
    else:
        all_button.config(text="AIO", bg='#4da89f')  # Change button text and color

# Initially hide the label
all_info_label.place_forget()

all_button = Button(root, text="AIO", font="Elephant 10", bg='#4da89f', activebackground='#1f534f', fg='#202930',
                    border=2, command=all_all)
all_button.place(x=320, y=294)

start_button = Button(root, text="Start Camera", font="Elephant 10", bg='#4da89f', activebackground='#1f534f',
                      fg='#202930', border=2, command=start_camera)
start_button.place(x=108, y=350)

root.mainloop()